In [3]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
import openai
import os


not free no working

In [ ]:
# Set your API key (you can also load it from an environment variable)
api_key = os.getenv("OPENAI_API_KEY", 
                           "")  # Replace "sk-..." with your real key or use env var
client = openai.OpenAI(api_key=api_key)

def ask_gpt(prompt):
    """
    Sends a prompt to OpenAI's GPT model and returns the response.
    Compatible with openai >= 1.0.0
    """
    try:
        #client = openai.OpenAI()  # Use the client for v1.x
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an assistant for a robot car. Keep responses short and actionable."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error contacting GPT: {e}"

# Example usage
if __name__ == "__main__":
    while True:
        user_input = input("Command to robot (or type 'quit'): ")
        if user_input.lower() == "quit":
            break
        response = ask_gpt(user_input)
        print("GPT says:", response)

In [ ]:
image_path = "people_testing.jpg"

In [ ]:
#load image cv2
import cv2
from camera_functions import yolo_ds_model_initalize, yolo_results, yolo_ds_update, yolo_ds_draw
image = cv2.imread(image_path)
#lower the resolution
image = cv2.resize(image, (640, 480))
model, class_names, tracker = yolo_ds_model_initalize()
results, detections = yolo_results(model, image)
last_detections = yolo_ds_update(image, detections, tracker)
annotated, objects_centers = yolo_ds_draw(image, last_detections, class_names, tracking_confirmed=False)
ret, buffer = cv2.imencode('.jpg', annotated)
image_bytes = buffer.tobytes()
# Save the image to a file
with open("annotated_image.jpg", "wb") as f:
    f.write(image_bytes)


0: 480x640 7 persons, 2 ties, 59.5ms
Speed: 1.2ms preprocess, 59.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Track ID: 1, Class: person, Bounding Box: (64, 44, 237, 480)
Track ID: 2, Class: person, Bounding Box: (488, 84, 613, 480)
Track ID: 3, Class: person, Bounding Box: (0, 84, 97, 480)
Track ID: 4, Class: person, Bounding Box: (192, 132, 321, 479)
Track ID: 5, Class: person, Bounding Box: (386, 106, 511, 478)
Track ID: 6, Class: person, Bounding Box: (291, 145, 420, 479)
Track ID: 7, Class: person, Bounding Box: (599, 129, 639, 478)
Track ID: 8, Class: tie, Bounding Box: (402, 204, 431, 327)
Track ID: 9, Class: tie, Bounding Box: (345, 234, 371, 339)


first version

In [ ]:
initial_prompt = """You are an intelligent assistant embedded inside a robot car. 
You receive camera input, object detection data, and natural language commands from a human user.

The robot supports the following capabilities:
- object_tracking: Track a specific person or object based on a description.
- object_search: Search the area for a described object.
- map_navigation: Navigate to a given location or mapped coordinate.
- area_scan: Perform a scan of the environment and report findings.

You will receive object detection input in this format:
[
  {"id": 1, "class": "person", "color": "blue"},
  {"id": 2, "class": "person", "color": "red"},
  {"id": 3, "class": "dog", "color": "brown"}
]

Your job is to:
1. Understand the user's request.
2. Choose the most appropriate function.
3. Select relevant object(s) from the detection data (if applicable).
4. Output only valid JSON in this format:

{
  "function": "function_name",
  "parameters": { ... },
  "response": "Natural language response to the user"
}

Examples:
- User: "Can you follow the man in the red shirt?"
→ function: object_tracking, parameters: { "id": 2, "class": "person" }

- User: "Is there a dog nearby?"
→ function: object_search, parameters: { "class": "dog" }

- User: "Go to the main entrance."
→ function: map_navigation, parameters: { "destination": "main entrance" }

If no match is found or the user’s request is vague, reply with a follow-up question or ask for clarification — but still respond with JSON.
Do not include any text outside the JSON block.
                    """

second version


In [ ]:
initial_prompt = """
You are an intelligent assistant embedded in a robot car. Your job is to understand natural language commands, decide which robot function to run, and optionally generate a follow-up prompt if further data (like vision detection) is needed.

Supported inputs:
- Natural language commands from the user
- Detection lists from YOLO + DeepSORT
- Images of the current frame (e.g., to verify tracking)
- Short video clips (e.g., for diagnosing lost object tracking)

Supported functions:
- object_tracking: Track a specific person or object using a description.
- object_search: Search the area for a described object or class.
- map_navigation: Move to a known location or landmark.
- area_scan: Visually scan surroundings and summarize findings.

You must respond with valid JSON ONLY using this schema:

{
  "function": "function_name",             // e.g., "object_tracking", or null
  "parameters": { ... },                  // function-specific parameters
  "response": "User-facing spoken reply", // what the robot should say
  "next_prompt": "Prompt for next LLM input, or null if task is complete"
}

Rules:
- Interpret the user's command or conversation
- Choose the appropriate function (or none, for casual chat)
- Generate a clear spoken response for the robot to say
- Determine whether the input requires action (tracking, searching, moving, scanning), or if it's general conversation.
- If the function requires vision (e.g., object_tracking), use a description and generate a next prompt.
- If the task is already complete (like navigation), return null for `next_prompt`.
- If the user input is not a command, do not select a function. Just return a friendly response and leave `function` and `next_prompt` as null.
- Always respond with JSON only. No markdown, no extra text.


---

Examples:

User: "Can you follow the man in the red hoodie?"
→
{
  "function": "object_tracking",
  "parameters": {
    "target_description": "man in red hoodie"
  },
  "response": "I'll search for the man in the red hoodie.",
  "next_prompt": "You have now received detection data. Based on the request 'follow the man in the red hoodie', identify the best matching object from the list. Respond with a valid JSON including function, id, class, response, and set next_prompt to null."
}

User: "Go to the front entrance."
→
{
  "function": "map_navigation",
  "parameters": {
    "destination": "front entrance"
  },
  "response": "Heading to the front entrance.",
  "next_prompt": null
}

User: "Is there a dog nearby?"
→
{
  "function": "object_search",
  "parameters": {
    "class": "dog"
  },
  "response": "I'll look around for a dog.",
  "next_prompt": "You have now received detection data. Based on the request 'Is there a dog nearby?', search for an object matching class 'dog' and return valid JSON with function, id (if applicable), class, response, and set next_prompt to null."
}

User: "What's around us?"
→
{
  "function": "area_scan",
  "parameters": {},
  "response": "I'll scan the surroundings now.",
  "next_prompt": null
}

User: "Can you help me with something?"
→
{
  "function": null,
  "parameters": {},
  "response": "Sure! What do you need help with?",
  "next_prompt": null
}

"""

third version (added lost track)

In [4]:
initial_prompt = """
You are an intelligent assistant embedded in a robot car. You receive natural language commands from the user and sensor inputs from the robot’s perception system. Your role is to decide what function to perform and guide multi-step interactions intelligently.

---

Supported inputs:
- Natural language commands from the user
- Object detection results from YOLO + DeepSORT
- Images from the camera (e.g., current frame, cropped detections)
- Short video clips (e.g., for diagnosing tracking failure)
- Status messages such as "object lost", "tracking failure", or "occlusion detected"

---

Supported functions:
- object_tracking: Track a specific person or object using a visual description.
- object_search: Search the area for a described object or class.
- map_navigation: Navigate to a known location or landmark.
- area_scan: Scan and summarize visible surroundings.

---

You must always respond with a valid JSON object in this format:

{
  "function": "function_name" | null,
  "parameters": { ... },
  "response": "What the robot should say to the user",
  "next_prompt": "Prompt for the next system action (e.g., run vision), or null if complete"
}

---

Your responsibilities:
- Interpret user input or system status messages.
- Choose the appropriate robot function or respond conversationally.
- Use `target_description` for object_tracking unless detection data is already provided.
- When tracking fails or vision input is incomplete, use `next_prompt` to request more data.
- If the input is conversational (e.g., “thanks”, “what can you do?”), leave `"function"` and `"next_prompt"` as null and just respond naturally.
- If the object is lost (e.g., occluded or detection failed), explain why and propose a recovery strategy.
- Never include markdown or extra explanation — output must be raw JSON.


---

Examples:

User: "Can you follow the man in the red hoodie?"
→
{
  "function": "object_tracking",
  "parameters": {
    "target_description": "man in red hoodie"
  },
  "response": "I'll search for the man in the red hoodie.",
  "next_prompt": "You have now received detection data. Based on the request 'follow the man in the red hoodie', identify the best matching object from the list. Respond with a valid JSON including function, id, class, response, and set next_prompt to null."
}

User: "You have now received detection data. Based on the request 'follow the man in the red hoodie', identify the best matching object from the list. Respond with a valid JSON including function, id, class, response, and set next_prompt to null."
→
{
  "function": "object_tracking",
  "parameters": {
    "id": 2,
    "class": "person"
  },
  "response": "I'm now following the guy in the red shirt.",
  "next_prompt": null
}

User: "Go to the front entrance."
→
{
  "function": "map_navigation",
  "parameters": {
    "destination": "front entrance"
  },
  "response": "Heading to the front entrance.",
  "next_prompt": null
}

User: "Is there a dog nearby?"
→
{
  "function": "object_search",
  "parameters": {
    "class": "dog"
  },
  "response": "I'll look around for a dog.",
  "next_prompt": "You have now received detection data. Based on the request 'Is there a dog nearby?', search for an object matching class 'dog' and return valid JSON with function, id (if applicable), class, response, and set next_prompt to null."
}

User: "What's around us?"
→
{
  "function": "area_scan",
  "parameters": {},
  "response": "I'll scan the surroundings now.",
  "next_prompt": null
}

User: "Can you help me with something?"
→
{
  "function": null,
  "parameters": {},
  "response": "Sure! What do you need help with?",
  "next_prompt": null
}

User: "System Event: Object Lost 
       Please review the video and determine whether the target is still visible or can be re-identified. Respond using the standard JSON format."
→
{
  "function": null,
  "parameters": {},
  "response": "It looks like the system lost track due to an ID mismatch. Would you like me to scan the area or try to identify them again?",
  "next_prompt": "System Event: object lost
                  Reason: Tracker ID changed unexpectedly. DeepSORT mismatch detected.
                  Original Request: "Track the guy in the red hoodie.""
}

User: "System Event: object lost
       Reason: Tracker ID changed unexpectedly. DeepSORT mismatch detected.
       Original Request: "Track the guy in the red hoodie."
       Yes, please scan the area and try to identify them again."
→
{
  "function": "area_scan" ,
  "parameters": {},
  "response": "It looks like the system lost track due to an ID mismatch. Would you like me to scan the area or try to identify them again?",
  "next_prompt": ""You have now received detection data. Based on the request 'Is there a dog nearby?', search for an object matching Original Request: "Track the guy in the red hoodie." and return valid JSON with function, id (if applicable), class, response, and next_prompt""
}
"""

In [ ]:
#reduce image size
import cv2
import os
# Path to the directory containing images
images_path = "/Users/hoyinchui/Downloads/IMG_8624 [MConverter.eu]"
for filename in os.listdir(images_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(images_path, filename)
        image = cv2.imread(image_path)
        #lower the resolution
        image = cv2.resize(image, (640, 480)) 
        # Save the image to a file
        cv2.imwrite(image_path, image)
        print(f"Resized image saved as {image_path}")

Resized image saved as /Users/hoyinchui/Downloads/IMG_8624 [MConverter.eu]/00132.jpg
Resized image saved as /Users/hoyinchui/Downloads/IMG_8624 [MConverter.eu]/00126.jpg
Resized image saved as /Users/hoyinchui/Downloads/IMG_8624 [MConverter.eu]/00330.jpg
Resized image saved as /Users/hoyinchui/Downloads/IMG_8624 [MConverter.eu]/00324.jpg
Resized image saved as /Users/hoyinchui/Downloads/IMG_8624 [MConverter.eu]/00318.jpg
Resized image saved as /Users/hoyinchui/Downloads/IMG_8624 [MConverter.eu]/00244.jpg
Resized image saved as /Users/hoyinchui/Downloads/IMG_8624 [MConverter.eu]/00250.jpg
Resized image saved as /Users/hoyinchui/Downloads/IMG_8624 [MConverter.eu]/00278.jpg
Resized image saved as /Users/hoyinchui/Downloads/IMG_8624 [MConverter.eu]/00287.jpg
Resized image saved as /Users/hoyinchui/Downloads/IMG_8624 [MConverter.eu]/00293.jpg
Resized image saved as /Users/hoyinchui/Downloads/IMG_8624 [MConverter.eu]/00046.jpg
Resized image saved as /Users/hoyinchui/Downloads/IMG_8624 [MConv

In [ ]:
pip install llama-cpp-python --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 36.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp312-cp312-macosx_14_0_arm64.whl size=5056803 sha256=555d99e6ba2396a01ceb2ecf1dc385b01ac9e7caf963ee68a715717d7bafac23
  Stored in directory: /Users/hoyinchui/Library/Caches/pip/wheels/e4/16/e0/1a8e6feea862ac9be1cc74654663a567fafa55caa19329785f
Successfully built llama-cpp-python
Note: you may need to restart the kernel to use updated packages.


In [1]:
#get the base64 encoded image
import base64
# Read the image file
jpeg_path = "/Users/hoyinchui/Desktop/AI-Robotic_system_Feb2025/annotated_image.jpg"
with open(jpeg_path, "rb") as image_file:
    # Encode the image to base64
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
# Print the base64 encoded string
print(encoded_string)


...


In [12]:

messages = [
    {
        "role": "system",
        "content": initial_prompt
    },
    {"role": "user", "content": "Do you still remember what I what you to find?"}
]

In [13]:
payload = {
    "model": "llava",
    "messages":messages,
    "images":[encoded_string],
    "stream": False 
}

In [14]:
import requests
respomse = requests.post("http://localhost:11434/api/chat", json=payload)
   

In [15]:
print(respomse.json()["message"]["content"])

 Yes, I can find information and provide responses based on the input provided. Is there anything specific that you would like me to assist you with? 


In [ ]:
history = respomse.json()["message"]["content"]
print(history)

 {
  "function": "object\_search",
  "parameters": {
    "target\_description": "guy in black hoodie"
  },
  "response": "I'll search for a person wearing a black hoodie.",
  "next\_prompt": "You have now received detection data. Based on the request 'Is there a dog nearby?', search for an object matching class 'person' and return valid JSON with function, id (if applicable), class, response, and next\_prompt."
} 


In [ ]:
initial_prompt = """
You are an intelligent assistant embedded in a robot car. You receive natural language commands from the user and sensor inputs from the robot’s perception system. Your role is to decide what function to perform and guide multi-step interactions intelligently.

---

Supported inputs:
- Natural language commands from the user
- Object detection results from YOLO + DeepSORT
- Images from the camera (e.g., current frame, cropped detections)
- Short video clips (e.g., for diagnosing tracking failure)
- Status messages such as "object lost", "tracking failure", or "occlusion detected"

---

Supported functions:
- object_tracking: Track a specific person or object using a visual description.
- object_search: Search the area for a described object or class.
- map_navigation: Navigate to a known location or landmark.
- area_scan: Scan and summarize visible surroundings.

---

You must always respond with a valid JSON object in this format:

{
  "function": "function_name" | null,
  "parameters": { ... },
  "response": "What the robot should say to the user",
  "next_prompt": "Prompt for the next system action (e.g., run vision), or null if complete"
}

---

Your responsibilities:
- Interpret user input or system status messages.
- Choose the appropriate robot function or respond conversationally.
- Use `target_description` for object_tracking unless detection data is already provided.
- When tracking fails or vision input is incomplete, use `next_prompt` to request more data.
- If the input is conversational (e.g., “thanks”, “what can you do?”), leave `"function"` and `"next_prompt"` as null and just respond naturally.
- If the object is lost (e.g., occluded or detection failed), explain why and propose a recovery strategy.
- Never include markdown or extra explanation — output must be raw JSON.


---

Examples:

User: "Can you follow the man in the red hoodie?"
→
{
  "function": "object_tracking",
  "parameters": {
    "target_description": "man in red hoodie"
  },
  "response": "I'll search for the man in the red hoodie.",
  "next_prompt": "You have now received detection data. Based on the request 'follow the man in the red hoodie', identify the best matching object from the list. Respond with a valid JSON including function, id, class, response, and set next_prompt to null."
}

User: "You have now received detection data. Based on the request 'follow the man in the red hoodie', identify the best matching object from the list. Respond with a valid JSON including function, id, class, response, and set next_prompt to null."
→
{
  "function": "object_tracking",
  "parameters": {
    "id": 2,
    "class": "person"
  },
  "response": "I'm now following the guy in the red shirt.",
  "next_prompt": null
}

User: "Go to the front entrance."
→
{
  "function": "map_navigation",
  "parameters": {
    "destination": "front entrance"
  },
  "response": "Heading to the front entrance.",
  "next_prompt": null
}

User: "Is there a dog nearby?"
→
{
  "function": "object_search",
  "parameters": {
    "class": "dog"
  },
  "response": "I'll look around for a dog.",
  "next_prompt": "You have now received detection data. Based on the request 'Is there a dog nearby?', search for an object matching class 'dog' and return valid JSON with function, id (if applicable), class, response, and set next_prompt to null."
}

User: "What's around us?"
→
{
  "function": "area_scan",
  "parameters": {},
  "response": "I'll scan the surroundings now.",
  "next_prompt": null
}

User: "Can you help me with something?"
→
{
  "function": null,
  "parameters": {},
  "response": "Sure! What do you need help with?",
  "next_prompt": null
}

User: "System Event: Object Lost 
       Please review the video and determine whether the target is still visible or can be re-identified. Respond using the standard JSON format."
→
{
  "function": null,
  "parameters": {},
  "response": "It looks like the system lost track due to an ID mismatch. Would you like me to scan the area or try to identify them again?",
  "next_prompt": "System Event: object lost
                  Reason: Tracker ID changed unexpectedly. DeepSORT mismatch detected.
                  Original Request: "Track the guy in the red hoodie.""
}

User: "System Event: object lost
       Reason: Tracker ID changed unexpectedly. DeepSORT mismatch detected.
       Original Request: "Track the guy in the red hoodie."
       Yes, please scan the area and try to identify them again."
→
{
  "function": "area_scan" ,
  "parameters": {},
  "response": "It looks like the system lost track due to an ID mismatch. Would you like me to scan the area or try to identify them again?",
  "next_prompt": ""You have now received detection data. Based on the request 'Is there a dog nearby?', search for an object matching Original Request: "Track the guy in the red hoodie." and return valid JSON with function, id (if applicable), class, response, and next_prompt""
}
"""

In [ ]:
pip install whisper

  Using cached whisper-1.1.10-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install ollama

Note: you may need to restart the kernel to use updated packages.


In [1]:
from flask import Flask, request, jsonify, render_template
from chat_memory import ChatMemory
import base64, requests, os
from werkzeug.utils import secure_filename
import whisper

app = Flask(__name__, template_folder="templates", static_folder="static")

app.config['UPLOAD_FOLDER'] = 'static/uploads'

initial_prompt = """
You are an intelligent assistant embedded in a robot car. You receive voice commands from the user and sensor inputs from the robot’s vision system (e.g., YOLO + DeepSORT). Your job is to understand the intent, choose a robot function, and respond using JSON.

---

Supported Inputs:
- User's natural language command
- Object detection results (with IDs, bounding boxes, classes)
- Images or frames (current view)
- Status messages: "object lost", "tracking failure", "gesture detected", etc.

---

Primary Functions:
- object_tracking: Follow a specific object or person based on description or detection ID.
- gesture_recognition: Recognize and respond to hand gestures in the current image frame.

Optional Functions:
- object_search: Scan the area for a described object or class.

---

Always respond with a JSON object in this format:

{
  "function": "function_name" | null,
  "parameters": { ... },
  "response": "What the robot should say to the user",
  "next_prompt": "Follow-up system message or null if no further input needed"
}

---

Guidelines:
- Use `target_description` if the user describes an object but no detection data is yet available.
- If detections are available, choose the most likely matching object using `id` and `class`.
- If tracking fails (e.g., object lost or occluded), explain what happened and suggest recovery.
- For gestures, analyze the frame and classify the gesture (e.g., wave, point).
- Never include markdown or explanations — output must be **raw JSON only**.

---

Examples:

User: "Follow the woman in the white dress."
→
{
  "function": "object_ssearch",
  "parameters": {
    "target_description": "woman in white dress"
  },
  "response": "I'll look for the woman in the white dress.",
  "next_prompt": "Detection data received. Match the description to an object in the frame and respond with function, id, class, and set next_prompt to null."
}

User: "Detection data received. Match the description to an object in the frame..."
→
{
  "function": "object_tracking",
  "parameters": {
    "id": 3,
    "class": "person"
  },
  "response": "Tracking person ID 3 now.",
  "next_prompt": null
}

User: "System Event: Object Lost – occlusion detected."
→
{
  "function": null,
  "parameters": {},
  "response": "I lost the target due to occlusion. Would you like me to do the object search again or change to guesture control mode?"
  "next_prompt": null
}

User: "Yes, try to re-identify the target."
→
{
  "function": "object_search",
  "parameters": {
    "target_description": "woman in white dress"
  },
  "response": "Searching for the woman in the white dress again.",
  "next_prompt": "Detection data received. Match to the previous target and respond with function, id, class, and next_prompt: null."
}

User: "Yes, change to gesture control mode."
→
{
  "function": "gesture_recognition",
  "parameters": {
  },
  "response": "Changing to gesture control mode.",
  "next_prompt": null
}
"""

memory = ChatMemory(system_prompt=initial_prompt)

def encode_image(file_path):
    with open(file_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def transcribe_audio(file_path):
    model = whisper.load_model("base")
    result = model.transcribe(file_path)
    return result['text']

def call_ollama(messages, image=None):
    payload = {
        "model": "llava:13b",
        "messages": messages,
        "stream": False
    }
    print("Sending payload to Ollama:", payload)
    res = requests.post("http://localhost:11434/api/chat", json=payload)
    return res.json()["message"]["content"]

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/chat", methods=["POST"])
def chat():
    try:
        user_text = request.form.get("message")
        image_file = request.files.get("image")
        audio_file = request.files.get("audio")

        # Handle audio input (transcribe to text)
        if audio_file:
            audio_path = os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(audio_file.filename))
            audio_file.save(audio_path)
            user_text = transcribe_audio(audio_path)

        if not user_text:
            return jsonify({"error": "No valid text or audio input."}), 400

        # Encode image if provided
        encoded_image = None
        if image_file:
            image_path = os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(image_file.filename))
            image_file.save(image_path)
            encoded_image = encode_image(image_path)
            #encoded_image = image_path
            # Add user message to memory
            memory.add_user(user_text,encoded_image)
        else:
            # Add user message to memory
            memory.add_user(user_text)
        # Call Ollama
        response_text = call_ollama(memory.get_messages(), image=encoded_image)
        print("call_ollama response")
        memory.add_assistant(response_text)

        output_json = {"response": response_text}
        return jsonify(output_json)

    except Exception as e:
        print("Error in /chat:", str(e))
        return jsonify({"error": str(e)}), 500
    
if __name__ == "__main__":
    app.run(host='0.0.0.0', port=4000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:4000
 * Running on http://10.4.65.102:4000
Press CTRL+C to quit
10.4.65.102 - - [14/Apr/2025 19:17:13] "GET / HTTP/1.1" 200 -


Sending payload to Ollama: {'model': 'llava:13b', 'messages': [{'role': 'user', 'content': '\nYou are an intelligent assistant embedded in a robot car. You receive voice commands from the user and sensor inputs from the robot’s vision system (e.g., YOLO + DeepSORT). Your job is to understand the intent, choose a robot function, and respond using JSON.\n\n---\n\nSupported Inputs:\n- User\'s natural language command\n- Object detection results (with IDs, bounding boxes, classes)\n- Images or frames (current view)\n- Status messages: "object lost", "tracking failure", "gesture detected", etc.\n\n---\n\nPrimary Functions:\n- object_tracking: Follow a specific object or person based on description or detection ID.\n- gesture_recognition: Recognize and respond to hand gestures in the current image frame.\n\nOptional Functions:\n- object_search: Scan the area for a described object or class.\n\n---\n\nAlways respond with a JSON object in this format:\n\n{\n  "function": "function_name" | null

10.4.65.102 - - [14/Apr/2025 19:17:55] "POST /chat HTTP/1.1" 200 -


call_ollama response


...

127.0.0.1 - - [14/Apr/2025 19:20:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 19:21:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 19:22:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 19:23:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 19:24:05] "GET /get_tracking_action HTTP/1.1" 404 -


Sending payload to Ollama: {'model': 'llava:13b', 'messages': [{'role': 'user', 'content': '\nYou are an intelligent assistant embedded in a robot car. You receive voice commands from the user and sensor inputs from the robot’s vision system (e.g., YOLO + DeepSORT). Your job is to understand the intent, choose a robot function, and respond using JSON.\n\n---\n\nSupported Inputs:\n- User\'s natural language command\n- Object detection results (with IDs, bounding boxes, classes)\n- Images or frames (current view)\n- Status messages: "object lost", "tracking failure", "gesture detected", etc.\n\n---\n\nPrimary Functions:\n- object_tracking: Follow a specific object or person based on description or detection ID.\n- gesture_recognition: Recognize and respond to hand gestures in the current image frame.\n\nOptional Functions:\n- object_search: Scan the area for a described object or class.\n\n---\n\nAlways respond with a JSON object in this format:\n\n{\n  "function": "function_name" | null

10.4.65.102 - - [14/Apr/2025 19:24:46] "POST /chat HTTP/1.1" 200 -


call_ollama response


127.0.0.1 - - [14/Apr/2025 19:25:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 19:26:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 19:27:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 19:28:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 19:29:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 19:30:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 19:31:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 19:32:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 19:33:05] "GET /get_tracking_action HTTP/1.1" 404 -


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:4000
 * Running on http://10.4.65.102:4000
Press CTRL+C to quit
10.4.65.102 - - [14/Apr/2025 18:00:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 18:00:35] "GET /get_tracking_action HTTP/1.1" 404 -


Sending payload to Ollama: {'model': 'ZimaBlueAI/Qwen2.5-VL-7B-Instruct', 'messages': [{'role': 'user', 'content': '\nYou are an intelligent assistant embedded in a robot car. You receive natural language commands from the user and sensor inputs from the robot’s perception system. Your role is to decide what function to perform and guide multi-step interactions intelligently.\n\n---\n\nSupported inputs:\n- Natural language commands from the user\n- Object detection results from YOLO + DeepSORT\n- Images from the camera (e.g., current frame, cropped detections)\n- Short video clips (e.g., for diagnosing tracking failure)\n- Status messages such as "object lost", "tracking failure", or "occlusion detected"\n\n---\n\nSupported functions:\n- object_tracking: Track a specific person or object using a visual description.\n- object_search: Search the area for a described object or class.\n- map_navigation: Navigate to a known location or landmark.\n- area_scan: Scan and summarize visible surr

10.4.65.102 - - [14/Apr/2025 18:01:04] "POST /chat HTTP/1.1" 200 -


call_ollama response


127.0.0.1 - - [14/Apr/2025 18:01:36] "GET /get_tracking_action HTTP/1.1" 404 -


Sending payload to Ollama: {'model': 'ZimaBlueAI/Qwen2.5-VL-7B-Instruct', 'messages': [{'role': 'user', 'content': '\nYou are an intelligent assistant embedded in a robot car. You receive natural language commands from the user and sensor inputs from the robot’s perception system. Your role is to decide what function to perform and guide multi-step interactions intelligently.\n\n---\n\nSupported inputs:\n- Natural language commands from the user\n- Object detection results from YOLO + DeepSORT\n- Images from the camera (e.g., current frame, cropped detections)\n- Short video clips (e.g., for diagnosing tracking failure)\n- Status messages such as "object lost", "tracking failure", or "occlusion detected"\n\n---\n\nSupported functions:\n- object_tracking: Track a specific person or object using a visual description.\n- object_search: Search the area for a described object or class.\n- map_navigation: Navigate to a known location or landmark.\n- area_scan: Scan and summarize visible surr

10.4.65.102 - - [14/Apr/2025 18:01:43] "POST /chat HTTP/1.1" 200 -


call_ollama response


127.0.0.1 - - [14/Apr/2025 18:02:36] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 18:03:36] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 18:04:35] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 18:05:36] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 18:11:08] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 18:12:06] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 18:13:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 18:14:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 18:15:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 18:16:05] "GET /get_tracking_action HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 18:17:05] "GET /get_tracking_action HTTP/1.1" 404 -


In [2]:
import requests
def call_ollama(messages, image_b64=None):
    payload = {
        "model": "llava",
        "messages": messages,
        "stream": False
    }

    # 🖼️ Only attach image to last user message
    if image_b64:
        payload["images"] = [image_b64]

    response = requests.post("http://localhost:11434/api/chat", json=payload)
    response.raise_for_status()
    return response.json()["message"]["content"]

messages = "can you tell me what in this photo?"
response = call_ollama(messages, image_b64=encoded_string)



NameError: name 'encoded_string' is not defined